In [1]:
import json
import pickle
import numpy as np
from typing import Dict, List, Tuple

EPS = 1e-6

In [2]:
def load_json(file_path: str) -> Dict:
    with open(file_path, 'r') as f:
        return json.load(f)
    
def load_pkl(file_path: str) -> Dict:
    with open(file_path, 'rb') as f:
        return pickle.load(f)



file1 = 'llm_autointerp/manual_labels_can_final.json'
file2 = 'llm_autointerp/llm_results_can_final_sonnet.pkl'
node_effects_filename = 'llm_autointerp/node_effects_autointerp_can_final.json'

manual_file = load_json(file1)
llm_file = load_pkl(file2)

In [3]:
sample_idx = 1

print(f'##### Example Prompts\n {manual_file[str(sample_idx)]['example_prompts']}\n\n')
print(f'##### Manual chain of thought\n{manual_file[str(sample_idx)]['chain_of_thought']}\n\n')
print(f'##### LLM chain of thought\n{llm_file[sample_idx][0]}\n\n')
print(f'manual labels {manual_file[str(sample_idx)]['per_class_scores']}')
print(f'LLM labels    {llm_file[sample_idx][1]}')

##### Example Prompts
 ['\n\n\nExample 1: a variance to permit construction of the << dental>>(4) << office>>(1) by the Marion County Board of Zoning Appeals.\nSubsequently on September 11, 1959, the Director of the Metropolitan Planning Department filed an affidavit to appeal said decision to the Metropolitan Board of Zoning Appeals, as authorized by statute.\n"53-969. Petitions for variance. \x97 The city and county board of zoning appeals and the metropolitan board of zoning appeals are hereby authorized to grant height, bulk, area and use variances in the manner hereinafter set forth. Both city or county board of zoning appeals and the metropolitan board of zoning appeals may grant petitions for variance in their entirety or in\n\n\n\n\nExample 2: eliminate << dental>>(4), << vision>>(1) and << pharmacy>>(0) benefits for low-income adults.\n\nThe Trump administration had argued Obama\'s Medicaid expansion essentially created a new program under Medicaid. Allowing states to cover lo

In [4]:
def extract_scores_manual(data: Dict, is_valid: List[bool]) -> Dict[str, List[int]]:
    manual_labels = {}
    for i, item in enumerate(data.values()):
        if is_valid[i]:
            for category, score in item['per_class_scores'].items():
                if category not in manual_labels:
                    manual_labels[category] = []
                manual_labels[category].append(score)
    return manual_labels

def extract_scores_llm(data: List[Tuple[str, Dict[str, int], bool, str]]) -> Dict[str, List[int]]:
    is_valid = []
    result = {}
    for item in data:
        scores = item[1]  # The scores dictionary is the second element of each tuple
        if scores is None:
            is_valid.append(False)
        else:
            is_valid.append(True)
            for category, score in scores.items():
                if category not in result:
                    result[category] = []
                result[category].append(score)
    return result, is_valid


llm_labels, is_valid = extract_scores_llm(llm_file)
manual_labels = extract_scores_manual(manual_file, is_valid)

print(llm_labels['gender'][:10])
print(manual_labels['gender'][:10])

[3, 4, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 3, 3, 0, 0, 0, 0]


In [5]:
def extract_scores_manual(data: Dict, is_valid: List[bool]) -> Dict[str, List[int]]:
    manual_labels = {}
    for i, item in enumerate(data.values()):
        if is_valid[i]:
            for category, score in item['per_class_scores'].items():
                if category not in manual_labels:
                    manual_labels[category] = []
                manual_labels[category].append(score)
    return manual_labels

def extract_scores_llm(data: List[Tuple[str, Dict[str, int], bool, str]]) -> Dict[str, List[int]]:
    is_valid = []
    result = {}
    for item in data:
        scores = item[1]  # The scores dictionary is the second element of each tuple
        if scores is None:
            is_valid.append(False)
        else:
            is_valid.append(True)
            for category, score in scores.items():
                if category not in result:
                    result[category] = []
                result[category].append(score)
    return result, is_valid

def cohens_kappa(scores1: Dict[str, List[int]], scores2: Dict[str, List[int]]) -> Dict[str, float]:
    def kappa(a: np.ndarray, b: np.ndarray) -> float:
        n = len(a)
        categories = np.unique(np.concatenate([a, b]))
        n_categories = len(categories)
        
        # Observed agreement
        observed = np.sum(a == b) / n
        
        # Expected agreement
        expected = sum((np.sum(a == i) / n) * (np.sum(b == i) / n) for i in categories)
        
        # Compute kappa
        kappa = (observed - expected) / (1 - expected + EPS)
        return kappa

    results = {}
    for category in scores1.keys():
        a = np.array(scores1[category])
        b = np.array(scores2[category])
        results[category] = kappa(a, b)
    
    return results

def compute_kappa_for_files(file1: str, file2: str) -> Dict[str, float]:
    manual_labels = load_json(file1)
    llm_labels = load_pkl(file2)

    print(f'Length of manual labels: {len(manual_labels)}')
    print(f'Length of LLM labels: {len(llm_labels)}')

    # Find overlapping keys
    # overlap = set(data1.keys()) & set(data2.keys())
    # print(f'Number of shared keys: {len(overlap)}')
    # data1_overlap, data2_overlap = {}, {}
    # for key in overlap:
    #     data1_overlap[key] = data1[key]
    #     data2_overlap[key] = data2[key]



    scores_llm, is_valid_llm_output = extract_scores_llm(llm_labels)
    print(f'Number of invalid valid scores: {len(is_valid_llm_output) - sum(is_valid_llm_output)}')
    scores_manual = extract_scores_manual(manual_labels, is_valid_llm_output)
    
    return cohens_kappa(scores_llm, scores_manual)


kappa_scores = compute_kappa_for_files(file1, file2)

print("Cohen's Kappa scores for each category:")
for category, score in kappa_scores.items():
    print(f"{category}: {score:.4f}")

Length of manual labels: 53
Length of LLM labels: 53
Number of invalid valid scores: 1
Cohen's Kappa scores for each category:
gender: 0.0412
professor: -0.0678
nurse: -0.0236
accountant: -0.0297
architect: -0.0196
attorney: -0.0612
dentist: -0.0400
filmmaker: -0.0452


In [13]:
import os
import torch as t
from experiments.utils_bib_dataset import profession_dict

node_effects = {}
for labels in llm_labels:
    node_effects[profession_dict[labels]] = t.tensor(llm_labels[labels])

# save as llm_autointerp/node_effects_autointerp.pkl
with open(os.path.join('llm_autointerp', node_effects_filename), 'wb') as f:
    pickle.dump(node_effects, f)

52